In [ ]:
from parse_tree import Node,Graph,Digraph,nohtml,Source
import nltk
from ipywidgets import *
from IPython.display import Image
import time

#Expr class to store each expression
class Expr(object):
    def __init__(self,i,a,sigma,tau,j,method):
        self.i = i
        self.a = a
        self.sigma = sigma
        self.method = method
        self.tau = tau
        self.j = j
    def __str__(self):
        return 's[' + str(self.i) + ']: ' + self.a + ' → ' + self.sigma + '•' + self.tau + ', ' + str(self.j)
    
class Animate(object):
    def __init__(self,grammar,x,auto_generate=False):
        #define an empty list to store the data used for animation
        self.graph_data = []
        result = self.parse(grammar,x)
        if len(result) == 0:
            print("Cannot accept this sentence!")
        #set this to TRUE to auto-generate the stages
        self.auto_generate = auto_generate
        #define a button
        self.btn = widgets.Button(description='next')
        display(self.btn)
        #iterator
        self.i = 0
        
    def parse(self,g: "grammar", x: "input"):
        global s
        n = len(x); x = '^' + x + '$'; S, π = g[0][0], g[0][2:]
        s = [{(S, '', π, 0)}] + [set() for _ in range(n)]#; print('   s[ 0 ]:', S, '→ •', π, ', 0')
        for i in range(n + 1):
            v = set() # visited items
            while v != s[i]:
                e = (s[i] - v).pop(); v.add(e) # pick an arbirary un-visited item
                A, σ, τ, j = e
                if len(τ) > 0 and τ[0] == x[i + 1]: # match, a == τ[0]
                    f = (A, σ + '(' + τ[0] + ')', τ[1:], j)
                    s[i + 1].add(f)
                    #print('M  s[', i + 1, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
                    #store animation data
                    self.graph_data.append(Expr(i+1, f[0], f[1], f[2], f[3], "Match"))
                elif len(τ) > 0: # predict, B == ω[0]
                    for f in ((r[0], '', r[2:], i) for r in g if r[0] == τ[0]):
                        s[i].add(f)
                        #print('P  s[', i, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
                        #store animation data
                        self.graph_data.append(Expr(i, f[0], f[1], f[2], f[3], "Predict"))
                else: # complete, len(τ) == 0
                    for f in ((B, μ + '(' + ν[0] + σ + ')', ν[1:], k) for (B, μ, ν, k) in s[j] if len(ν) > 0 and ν[0] == A):
                        s[i].add(f)
                        #print('C  s[', i, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
                        #store animation data
                        self.graph_data.append(Expr(i, f[0], f[1], f[2], f[3], "Complete"))
        return {f[1] for f in s[n] if f[0] == S}
    
    def gen_img(self):
        #generate png image file for each step
        #Expr(i,a,sigma,tau,j,method)
        for i in range(len(self.graph_data)):
            t_str = '('+self.graph_data[i].a+self.graph_data[i].sigma+')'
            t = nltk.Tree.fromstring(t_str)
            src = Graph(t).draw()
            g = Source(src,filename=str(i))
            g.render(format='png')
            
    def func(self,btn):
        if self.i < len(self.graph_data):
            item = self.graph_data[self.i]
            print("\x1b[91m"+item.method+"\x1b[0m")
            print(item)
            display(Image(filename=str(self.i)+'.png'))
            self.i += 1
        else:
            print("===FINISH===")
            
    def display(self):
        self.gen_img()
        if self.auto_generate:
            for i in range(len(self.graph_data)+1):
                time.sleep(1)
                self.func(self.btn)
        else:
            self.btn.on_click(self.func)

### TEST CASE 0 (Error Test Case)

In [ ]:
g1 = ("S→E", "E→a", "E→a+E")
x1 = "a+a+"

a1 = Animate(g1,x1)
a1.display()

### TEST CASE 1

In [ ]:
g1 = ("S→E", "E→a", "E→a+E")
x1 = "a+a+a"

a1 = Animate(g1,x1,auto_generate=True)
a1.display()

### TEST CASE 2

In [ ]:
g2 = ("S→E", "E→a", "E→a+E")
x2 = "a+a"

a2 = Animate(g2,x2)
a2.display()

### TEST CASE 3

In [ ]:
g3 = ("S→E", "E→a", "E→E+E")
x3 = "a+a+a"

a3 = Animate(g3,x3,auto_generate=True)
a3.display()

### TEST CASE 4

In [ ]:
g4 = ("S→E", "E→T", "E→E+T", "T→F", "T→T×F", "F→a")
x4 = "a+a×a"

a4 = Animate(g4,x4,auto_generate=True)
a4.display()

### TEST CASE 5

In [ ]:
g5 = ("S→E", "E→F", "E→E+F", "E→E-F", "F→a")
x5 = "a-a+a"

a5 = Animate(g5,x5,auto_generate=True)
a5.display()